_To use this notebook you must have a (free) FEO account, available [here](https://feo.transitionzero.org/). You may then install the feo-client library, if necessary, and then use the package to log in. You can also run notebook in a Colab notebook via the badge below._

<a target="_blank" href="https://colab.research.google.com/github/transition-zero/feo-client-examples/blob/main/feo-client-examples/1_assets.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# !pip install --extra-index-url https://test.pypi.org/simple/ feo-client==0.0.1a9

%env AUTH0_DOMAIN=dev-feo-tz.eu.auth0.com
%env FEO_API_URL=https://staging.api.feo.transitionzero.org
%env AUTH0_AUDIENCE=https://staging.api.feo.transitionzero.org
%env AUTH0_CLIENT_ID=bjzlyhLEClss31537pW3lu3Fkg97TzZ8 
%env FEO_API_VERSION=v1



env: AUTH0_DOMAIN=dev-feo-tz.eu.auth0.com
env: FEO_API_URL=https://staging.api.feo.transitionzero.org
env: AUTH0_AUDIENCE=https://staging.api.feo.transitionzero.org
env: AUTH0_CLIENT_ID=bjzlyhLEClss31537pW3lu3Fkg97TzZ8
env: FEO_API_VERSION=v1


In [2]:
from feo.client.auth import login

login()

1. In a browser navigate to:  https://dev-feo-tz.eu.auth0.com/activate?user_code=FMGZ-QBJX
2. Enter the following code:  FMGZ-QBJX
Checking for authentication.Authenticated!
- Id Token: eyJhbGciOi...


# Land Cover Data
_Object-oriented interface for Land Cover Data via the Python client_

## Land Availability for Renewable Energy

The amount of land available to build renewables is a key constraint on the energy transition and hence a key input to energy system models. Understanding where and how much of a given technology can be built, constrains how renewable capacity can be deployed and has downstream effects on the cost and emissions of the energy system. If we want to understand the energy transition, getting land availability right matters!

To highlight this, let's do a back of the envelope calculation to see how land availability impacts how much solar PV we can be deployed in the Indonesian province Jawa Tengah. We'll consider two scenarios. In the first scenario, 1% of land in Jawa Tengah is available for solar PV. In the second, only 0.05% is available.

In [3]:
# Assumptions
IDN_JT_land_area = 34290 # km2
scenario_1_percent = 0.01
scenario_2_percent = 0.005
solar_pv_density = 141 # MW/km2

# Compute UK solar PV capacity in MW
uk_solar_pv_capacity_scenario_1 = (IDN_JT_land_area * solar_pv_density * scenario_1_percent) 
uk_solar_pv_capacity_scenario_2 = (IDN_JT_land_area * solar_pv_density * scenario_2_percent) 
capacity_difference = uk_solar_pv_capacity_scenario_1 - uk_solar_pv_capacity_scenario_2

# Print results
print(f'Jawa Tengah solar PV capacity scenario 1: {uk_solar_pv_capacity_scenario_1/ 1000:.2f} GW')
print(f'Jawa Tengah solar PV capacity scenario 2: {uk_solar_pv_capacity_scenario_2/ 1000:.2f} GW')
print(f'Capacity difference: {capacity_difference/1000:.2f} GW')

Jawa Tengah solar PV capacity scenario 1: 48.35 GW
Jawa Tengah solar PV capacity scenario 2: 24.17 GW
Capacity difference: 24.17 GW


We see a difference of 24.17 GW of solar PV capacity between the two scenarios! 

To put this in context, IRENA estimates that the UK currently has 14.4 GW of solar PV capacity. So, the difference between these two scenarios is almost double the current solar PV capacity of the UK. This is a huge difference and highlights the importance of getting land availability right.

## Land Cover Data in FEO

A key input to land availability calculations is land cover data. Land cover data tells us what type of land is present in a given location. 

In FEO, we have aggregated land cover data into $5$ land type categories:

- `urban`: Urban areas
- `cropland`: Agricultural land
- `forest`: Forested land
- `bare`: Bare land
- `shrub`: Shrub land + Herbaceous land

These land categories are derived from Coperniucs Global Land Cover classes

The area of each land type category can be accessed with the `RecordCollection` client at the node level.

In [4]:
from feo.client import RecordCollection

forest_area = RecordCollection.search(node_id = 'IDN-JT', datum_detail='forest_landuse') 
forest_area

,node_id,public,source_id,source_node_id,target_node_id,timestamp,valid_timestamp_start,valid_timestamp_end,datum_type,datum_detail,value,unit,properties,id
0,IDN-JT,True,68,None,None,2023-11-28 16:02:49.271381,1850-01-01,2250-01-01,land_cover,forest_landuse,15612.464685,km2,{},656207


The client returns a pandas dataframe and the area can be found in the `value` column. Looking at the `unit` column we see it is given in $km^2$.

Let's compare how much crop land the is in Jawa Tengah compared to forest..

In [30]:
IDN_JT_forest = forest_area['value'].item()
IDN_JT_crop =  13207 #RecordCollection.search(node_id = 'IDN-JT', datum_detail='cropland_landuse')['value'].item()

print(f'Jawa Tengah forest area: {IDN_JT_forest:.2f} km2')
print(f'Jawa Tengah cropland area: {IDN_JT_crop:.2f} km2')

Jawa Tengah forest area: 15612.46 km2
Jawa Tengah cropland area: 13207.00 km2


.. and how much forest there in Jawa Tengah, compared to a neighboring province, Jawa Barat.

In [31]:
IDN_JB_forest = RecordCollection.search(node_id = 'IDN-JB', datum_detail='forest_landuse')['value'].item()

print(f'Jawa Tengah forest area: {IDN_JT_forest:.2f} km2')
print(f'Jawa Barat forest area: {IDN_JB_forest:.2f} km2')

Jawa Tengah forest area: 15612.46 km2
Jawa Barat forest area: 21384.93 km2


## Renewable Potentials


This land cover data can be used for early-stage mapping of land availability for renewables and to gain a rough estimate of renewable energy potentials at a given node. A key assumption for this estimation is the percentage of a given land type that we want to make available for a given technology. This should be informed by the land use priorities of the region, renewable resource information (how windy/sunny is the region?) and the land use requirements of the technology.

For example, let's estimate how changing our forest land use policy impacts how much solar PV capacity could be deployed in currently forested areas of Jawa Tengah. 

In [36]:
# Assumptions

solar_pv_density = 141 # MW/km2
less_deforestation_percentage = 0.01 # 1% of forest landuse is used for solar PV
more_deforestation_percentage = 0.05 # 5% of forest landuse is used for solar PV

IDN_JT_forest_pv_capacity_1 = (forest_area['value'].item() * solar_pv_density * less_deforestation_percentage)
IDN_JT_forest_pv_capacity_2 = (forest_area['value'].item() * solar_pv_density * more_deforestation_percentage)
print(f'If {less_deforestation_percentage*100}% of forest is used for solar PV in Jawa Tengah, then we can install {IDN_JT_forest_pv_capacity_1/1000:.2f} GW of solar PV in currently forested areas of Jawa Tengah.')
print(f'If we use {more_deforestation_percentage*100}%, we can install {IDN_JT_forest_pv_capacity_2/1000:.2f} GW.')


If 1.0% of forest is used for solar PV in Jawa Tengah, then we can install 22.01 GW of solar PV in currently forested areas of Jawa Tengah.
If we use 5.0%, we can install 110.07 GW.


## Coming Soon ....

Currently FEO is in it's beta phase and we are working on adding new features. In the future we will add the following features:

## References

- [Cox et al., 2018] Cox, Sarah L., Anthony J. Lopez, Andrea C. Watson, Nicholas W. Grue, and Jennifer E. Leisch. Renewable energy data, analysis, and decisions: a guide for practitioners. No. NREL/TP-6A20-68913. National Renewable Energy Lab.(NREL), Golden, CO (United States), 2018.